In [ ]:
import json
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance
import random
import numpy.lib.recfunctions as rfn
from random import randint

In [ ]:
qa = np.genfromtxt("uid_qa.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U128'),])

In [ ]:
qa.dtype.names

In [ ]:
fe = np.genfromtxt("uid_pre_elim.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U256'),])

In [ ]:
fe.dtype.names = ('uniqueID', 'bn', 'qn', 'pre_que', 'curr_elim')

fe.dtype.names
len(fe)


In [ ]:
#ftd = np.genfromtxt("face_id_descr.txt", delimiter = ";", skip_header = 1 , usecols = np.arange(0,2), dtype = [('U16'), ('U2056')])

In [ ]:
#ftd.dtype.names = ('img_id', 'description')

In [ ]:
#print(fe)

In [ ]:
## first convert strings to list of rows equal to rows in fe
def mk_list(str_vector):
    ##des_byid = str_vector
    all_x = []
    for i in str_vector:
        j = i.split()
        all_x.append(j)

    return all_x


In [ ]:
## all x is input of all img ids(s) before question
all_x = mk_list(fe['pre_que'])


In [ ]:
bl =[]

for img,q in zip(enumerate(all_x), mk_list(qa['que'])):
    if (len(img[1]) <2 or len(q) < 2) and img[0] not in bl:
        bl.append(img[0])
print(bl)


In [ ]:
len(all_x)
len(mk_list(qa['que']))

In [ ]:
all_x_clean = []
original_ind = []
bl_ind = []

for rown in range(len(all_x)):
    if rown in bl:
        bl_ind.append(rown)
    else:
        original_ind.append(rown)
        all_x_clean.append(all_x[rown])
        
len(all_x_clean)

        

In [ ]:
all_x_clean[0]

In [ ]:
yp_all = []
for i in all_x_clean:
    np.random.seed(len(i)*len(all_x_clean[(all_x_clean.index(i))-1])*len(i))
    yp_all.append(list(np.random.random_sample((len(i),))))
print(len(yp_all))

In [ ]:
all_cheat1 = mk_list(fe['curr_elim'])
all_cheat = [all_cheat1[i] for i in original_ind]

In [ ]:
yp_all[0]

In [ ]:

##for i,j in cheat_code:
##   print(i,j)

In [ ]:
a =np.array([1,2,4,5,6, 11, 20, 3])

a.argsort()[::-1][0:randint(1, len(a)-1)]

In [ ]:
y_top = []
for i,j in zip(yp_all, all_cheat):
    y_top.append(list(np.array(i).argsort()[::-1][0:randint(1,len(i)-1)]))
len(y_top)

In [ ]:
y_top[0:4]

In [ ]:
def mk_imgid(arr):
    y_pred = []
    for i,j in arr:
        yp = []
        for s in j:
            yp.append(i[s])
        y_pred.append(yp)
    return y_pred
    

In [ ]:
pred_img = mk_imgid(zip(all_x_clean, y_top))

In [ ]:
#print(list(check_acc))

In [ ]:
#for a,p in check_acc:
#    print(a,p)

In [ ]:
pre_all= [len(set(a).intersection(set(p)))/len(set(p)) if len(p) > 0 else 0 for a,p in zip(all_cheat, pred_img)]
rec_all = [len(set(a).intersection(set(p)))/len(set(a)) if len(a) > 0 else 0 for a,p in zip(all_cheat, pred_img)]

print(len(pre_all))
print(len(rec_all))

In [ ]:
fs_all= [(0.5*0.5 + 1)*pr*re/ (0.5*0.5*pr + re) if pr > 0 or re >0 else 0 for pr, re in zip(pre_all, rec_all)]
len(fs_all)

In [ ]:
pre_mean = np.mean(np.array(pre_all))
rec_mean = np.mean(np.array(rec_all))
fs_mean = np.mean(np.array(fs_all))
pre_mean

In [ ]:

op = ["20", "Random chance model", "default", "nothing",str(round(pre_mean,2)), str(round(rec_mean,2)), str(round(fs_mean,2))]

In [ ]:
with open('Results with description.txt', 'a') as f:
    f.write('; '.join(op)+ '\n')